In [1]:
import pandas as pd
import requests
import numpy as np

class Location:
  def __init__(self, lat, lon):
        self.lat = lat
        self.lon = lon

class Stop:
  def __init__(self, loc, drop, id):
        self.loc = loc
        self.drop = drop
        self.id = id

class Vehicle:
  def __init__(self, id, status, loc, route, max_capacity, current_capacity, volume, current_volume, consignments):
        self.id = id
        self.status = status
        self.loc = loc
        self.route = route
        self.max_capacity = max_capacity
        self.current_capacity = current_capacity
        self.volume = volume
        self.current_volume = current_volume
        self.consignments = consignments

class Consignment:
    def __init__(self, id, status, pickup_location, drop_location, pickup_date, drop_date, pickup_time, drop_time, weight, volume):
        self.id = id
        self.status = status
        self.pickup_location= pickup_location
        self.drop_location = drop_location
        self.pickup_date = pickup_date
        self.drop_date = drop_date
        self.pickup_time = pickup_time
        self.drop_time = drop_time
        self.weight = weight
        self.volume = volume

class Route:
    def __int__(self, start_point, end_point, sequence_of_points):
        self.start_point = start_point
        self.end_point = end_point
        self.sequence_of_points = sequence_of_points

In [2]:
def distance_matrix(coords):
    #calculates distance matrix for the given array of coords
    url = 'http://router.project-osrm.org/table/v1/driving/'
    for c in range(len(coords)) :
        lng = str(coords[c].lon)
        lat = str(coords[c].lat)
        if c == 0 :
            url = url+lng+','+lat
        else :
            url = url +';'+lng + ',' + lat
    
    url = url+'?annotations=distance'
     
    global distMat
    distMat = requests.get(url).json()['distances'];
    return distMat

In [3]:
def get_point_point_dist(a, b):
    global coords
    global distMat
    # get from distance matrix generated using osrm
    i=0
    j=0
    for c in range(len(coords)) :
        lng = coords[c].lon #longitude
        lat = coords[c].lat #latitude
        if(a.lon == lng & a.lat ==lat) :
            i=c
        if(b.lon == lng & b.lat == lat):
            j=c
    return distMat[i][j]

In [4]:
def can_be_allocated(consignment, vehicle):
    if consignment.weight > vehicle.current_capacity:
        return False
    if consignment.volume > vehicle.current_volume:
        return False
    return True

In [5]:
def get_polyline_point_dist(route, point):
    
    return True

In [6]:
def allocate_consignments_to_empty_vehicles(vehicles_list, consignment_list):
    local_consignment_list = consignment_list[:]
    for c in consignment_list:
        c_vehicles = vehicles_list
        nearest_vehicle = c_vehicles[0]
        min_distance = get_point_point_dist(c.pickup_location, nearest_vehicle.location)
        for vehicle in c_vehicles:
            distance = get_point_point_dist(c.pickup_location, vehicle.location)
            if distance < min_distance:
                nearest_vehicle = vehicle
                min_distance = distance
        c_vehicles.remove(nearest_vehicle)
        c_allocated = False
        while not c_allocated:
            if can_be_allocated(c, nearest_vehicle):
                c_allocated = True
                local_consignment_list.remove(c)
                nearest_vehicle.consignments.append(c.id)
                nearest_vehicle.current_capacity -= c.weight
                nearest_vehicle.current_volume -= c.volume
            else:
                nearest_vehicle = c_vehicles[0]
                min_distance = get_point_point_dist(c.pickup_location, nearest_vehicle.location)
                for vehicle in c_vehicles:
                    distance = get_point_point_dist(c.pickup_location, vehicle.location)
                    if distance < min_distance:
                        nearest_vehicle = vehicle
                        min_distance = distance
                c_vehicles.remove(nearest_vehicle)
            if not c_vehicles:
                break
    return local_consignment_list

In [7]:
def constraints_check(routewith_i,j):
  if(j.drop):
    return 1
  for k in routewith_i.sequence_of_points:
    if(k.drop and k.id==j):
        return 0
  return 1

In [8]:
def CVRP(V): # V - vehicle
    global consignment_list
    savingslist = []
    all_stops = []
    for consignment_id in V.consignments:
        for c in consignment_list:
            if c.id == consignment_id:
                all_stops.append(c.pickup_location)
                all_stops.append(c.drop_location)
                break

    for i in all_stops:
        for j in all_stops:
            savingslist.append([get_point_point_dist(i.end_point, V.loc) + get_point_point_dist(V.loc, j.start_point) - get_point_point_dist(i.end_point, j.start_point), (i, j)])
    savingslist.sort(reverse = True, key = lambda x: x[0])
    
    for l in savingslist:
        l[1][0]=i
        l[1][1]=j
        
        j_route=0
        i_route=0
        
        routewith_i=Route()
        routewith_j=Route()
        route_ij=Route()
        
        for k in all_routes:
            if(i in k.sequence_of_points):
                i_route=1
                routewith_i=k
                break
        for k in all_routes:
            if(j in k.sequence_of_points or (i==k.start_point) or (i==k.end_point)):
                j_route=1
                routewith_j=k
                break
        
        #case1 no routes assigned for both
        if(not (i_route or j_route)):
            if(i.id==j.id and not (i.drop)):    
                route_ij.start_point=i
                route_ij.end_point=j
                route_ij.sequence_of_points.append(i)
                route_ij.sequence_of_points.append(j)
            else:
                continue
        
        #case 2
        if(i_route ^ j_route):
            if(i_route):
                if(routewith_i.end_point==i and constraints_check(routewith_i,j)):        
                    routewith_i.end_point=j
                    routewith_i.sequence_of_points.append(j)
                    route_ij=routewith_i.copy()
                else:
                    continue      
            elif(j_route):
                if(routewith_j.start_point==j):
                    routewith_j.start_point=i
                    routewith_j.sequence_of_points.insert(0,i) 
                    route_ij=routewith_j.copy()
                    
        #3
        if(i_route and j_route):
            if(routewith_i.end_point==i and routewith_j.start_point==j):
                for k in routewith_j.sequence_of_points:
                    if(not constraints_check(routewith_i,k)):
                        a=0
                        break
                if a == 0:
                    continue 
                else:
                    routewith_i.end_point=routewith_j.end_point
                    routewith_i.sequence_of_points.extend(routewith_j.sequence_of_points)
                    route_ij=routewith_i.copy()
    current_tour.append(route_ij)

    return current_tour

In [9]:
def get_shortest(distance_list):
    shortest = min(distance_list, key = lambda x: x[0])
    return shortest[1]

In [10]:
def allocate_consignment(vehicles_list, consignment_id):
    return True

In [11]:
def route_alloc(vehicles_list, consignment_list):
    all_empty = True
    moving_vehicles = []
    unassigned_consignments_list = consignment_list
    for vehicle in vehicles_list:
        if not vehicle.consignments:
            all_empty = False
        if vehicle.status == True:
            moving_vehicles.append(vehicle)
    if all_empty and not moving_vehicles:
        unassigned_consignments_list = allocate_consignments_to_empty_vehicles(vehicles_list, consignment_list)

    while unassigned_consignments_list:
        c = unassigned_consignments_list[0]
        if len(moving_vehicles) != len(vehicles_list):
            distance_list = []
            for vehicle in vehicles_list:
                if vehicle.route:
                    distance_list.add(get_polyline_point_dist(vehicle.route, c.pickup), vehicle)
                else:
                    distance_list.add(get_polyline_point_dist(vehicle.loc, c.pickup), vehicle)
            vehicle_to_be_assigned = get_shortest(distance_list)
            if vehicle_to_be_assigned.status == True:
                allocate_consignment(moving_vehicles, c.id)
                #   vehicle_list
        else:
            allocate_consignment(vehicles_list, c.id)
            #   vehicle_list

In [12]:
vehicle_list = []
consignment_list = []

inptfile = open("inpt.txt", 'r')
inpt = inptfile.readlines()

vn = int(inpt[0])
cn = int(inpt[1])
base = 2
for i in range(vn):
    id = int(inpt[base + i*10])
    status = (inpt[base + i*10+2].strip() == "Idle")
    lat = float(inpt[base + i*10 + 3])
    lon = float(inpt[base + i*10 + 4])
    route = list(map(int, inpt[base + i*10 + 5].strip().split()))[1:]
    capacity = float(inpt[base + i*10 + 6])*1000
    volume = float(inpt[base + i*10 + 7])
    curr_capacity = float(inpt[base + i*10 + 8])*1000
    vehicle_list.append(Vehicle(id, status, Location(lat, lon), route, capacity, curr_capacity, volume, volume, route))

base = base + vn*10
for i in range(cn):
    id = int(inpt[base + i*13])
    status = int(inpt[base + i*13 + 1])
    plat = float(inpt[base + i*13 + 2])
    plon = float(inpt[base + i*13 + 3])
    dlat = float(inpt[base + i*13 + 4])
    dlon = float(inpt[base + i*13 + 5])
    pdate = inpt[base + i*13+6].strip()
    ddate = inpt[base + i*13+7].strip()
    ptimes = inpt[base + i*13+8].strip()
    ptimee = inpt[base + i*13+9].strip()
    dtimes = inpt[base + i*13+10].strip()
    dtimee = inpt[base + i*13+11].strip()
    weight = float(inpt[base + i*13 + 12])
    consignment_list.append(Consignment(id, status, Location(plat, plon), Location(dlat, dlon), pdate, ddate, (ptimes, ptimee), (dtimes, dtimee), weight, weight))

In [13]:
coords = [x.loc for x in vehicle_list]
coords.extend([x.pickup_location for x in consignment_list])
coords.extend([x.drop_location for x in consignment_list])

# import folium

# map = folium.Map()

# for c in coords:
#     folium.Marker(location = (c.lon, c.lat)).add_to(map)
#     print(c.lat, c.lon)

distMat = distance_matrix(coords)
distMat

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2152908, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2152908, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2152908, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2152908, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2152908, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2152908, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2152908, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2152908, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2152908, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2152908, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2152908, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2152908, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2152908, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2152908, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2152908, 0],
 [2153079.2,
  2153079.2,
  2153079.2,
  2153079.2,
  2153079.2,
  2153079.2,
  2153079.2,
  2153079

In [14]:
route_alloc(vehicle_list, consignment_list)

KeyboardInterrupt: 

In [ ]:
# vl = pd.read_csv("vl.csv").to_numpy()
# vehicle_list = []
# for i in vl:
#     vehicle_list.append(Vehicle(i[0], False, Location(i[1], i[2]), [], i[3], i[3], i[3], i[3], []))

In [ ]:
# cl = pd.read_csv("cl.csv").to_numpy()
# consignment_list = []
# for i in cl:
#     consignment_list.append(Consignment(i[0], Location(i[3], i[2]), Location(i[5], i[4]), 0, 0, 0, 0, i[1], i[1]))